# Teamfight Tactics - Set 2: Top Player Analysis

An analysis of the top 200 TFT players for each region at the end of Set 2.

This notebook brings all of the data sources together and formats them so that they are ready for further analysis.

## Data Sources

### tft.db
 - **players**: Data for the top 200 players for each region acquired from [lolchess.gg](https://lolchess.gg/)
 - **matches**: Data for 10 matches for each of the top players acquired from [lolchess.gg](https://lolchess.gg/)

## Imports

In [ ]:
import sqlite3
import csv
import pickle
from datetime import datetime, time
from pathlib import Path

## Input/Output file paths and directories

In [ ]:
today = datetime.today()
db_path = Path.cwd() / ".." / "data" / "raw" / "tft.db"
matches_pkl_traits_path = Path.cwd() / ".." / "data" / "interim" / "matches_traits.pkl"
matches_pkl_units_path = Path.cwd() / ".." / "data" / "interim" / "matches_units.pkl"
players_processed_csv_path = Path.cwd() / ".." / "data" / "processed" / "players.csv"
matches_processed_csv_path = Path.cwd() / ".." / "data" / "processed" / "matches.csv"

## Retrieve the Data from the database

In [3]:
# Initialise a connection to the database
conn = sqlite3.connect(db_path)
cur = conn.cursor()

In [4]:
# Get all of the data from the players table
cur.execute('SELECT * FROM players')
players = cur.fetchall()

# Get the column names from the players table
players_cols = [tuple[0] for tuple in cur.description]

In [5]:
# Get all of the data from the matches table
cur.execute('SELECT * FROM matches')
matches = cur.fetchall()

# Get the column names from the matches table
matches_cols = [tuple[0] for tuple in cur.description]

In [6]:
# The traits, units and length fields from the matches table are all
# of the wrong type. So we will store them separately to make changing
# their type easier later on.

cur.execute('SELECT traits FROM matches')
traits_pkl = cur.fetchall()

cur.execute('SELECT units FROM matches')
units_pkl = cur.fetchall()

cur.execute('SELECT length FROM matches')
match_length = cur.fetchall()

## Format the Data

The data from both tables in the database needs to be properly formatted before it can be used for the analysis and then output to a new file. It's important that the raw data remains unchanged during the formatting of the data. The following tasks need to be performed:
- **Column Heading Preparation**
    - Separate multiple words with "_"
    - Remove leading and trailing whitespace
    - Convert to lowercase
- **Data Preparation**
    - Make sure all data is in the type expected/required
    - Manipulate data into necessary format

### Column Heading Preparation

#### Separate multiple words with "_"

In [7]:
players_cols[0] = 'player_id'
players_cols[5] = 'win_rate'

matches_cols[0] = 'match_id'
matches_cols[1] = 'player_id'

#### Remove leading and trailing whitespace

In [8]:
players_cols = [column.strip() for column in players_cols]
matches_cols = [column.strip() for column in matches_cols]

#### Convert to lowercase

In [9]:
players_cols = [column.lower() for column in players_cols]
matches_cols = [column.lower() for column in matches_cols]

### Data Preparation

#### Make sure all data is in the type expected/required

1 - Define a function to make the checking of the data and outputting any discrepancies much easier:

In [10]:
def check_type(var, expected):
    '''
    Check that the variable is of the expected type
    and if it isn't then print out the type mismatch
    '''
    if not isinstance(var, expected):
        print('TYPE MISMATCH: {0} is not type {1}'.format(var, expected))

2 - The traits and units in the matches table were pickled before being stored in the database. They need to be unpickled to make the type checking possible:

In [11]:
traits = []
for row in traits_pkl:
    with open(matches_pkl_traits_path, 'wb') as traits_pkl_file:
        traits_pkl_file.write(row[0])

    with open(matches_pkl_traits_path, 'rb') as traits_pkl_file:
        traits.append(pickle.load(traits_pkl_file))

units = []
for row in units_pkl:
    with open(matches_pkl_units_path, 'wb') as units_pkl_file:
        units_pkl_file.write(row[0])
    
    with open(matches_pkl_units_path, 'rb') as units_pkl_file:
        units.append(pickle.load(units_pkl_file))

3 - The match length was stored in the database as a string but it should be of type 'time'. Convert it to the correct type:

In [12]:
lengths = []
for row in match_length:
    lengths.append(datetime.strptime(row[0], '%M:%S').time())

4 - Store all of the match data in a new list called 'processed_matches' with the 'length', 'traits' and 'units' variables all being replaced by the newly corrected types:

In [13]:
processed_matches = []
for i in range(len(matches)):
    processed_matches.append([
                                matches[i][0], 
                                matches[i][1],
                                matches[i][2],
                                matches[i][3],
                                lengths[i],
                                traits[i],
                                units[i]
                            ])

5 - Check the types for each field in the players data:

In [14]:
# player_id
check_type(players[0][0], int)

# rank
check_type(players[0][1], int)

# name
check_type(players[0][2], str)

# tier
check_type(players[0][3], str)

# lp
check_type(players[0][4], int)

# win_rate
check_type(players[0][5], float)

# played
check_type(players[0][6], int)

# wins
check_type(players[0][7], int)

# losses
check_type(players[0][8], int)

# region
check_type(players[0][9], str)

6 - Check the types for each field for the matches data:

In [15]:
# match_id
check_type(processed_matches[0][0], int)

# player_id
check_type(processed_matches[0][1], int)

# placement
check_type(processed_matches[0][2], int)

# mode
check_type(processed_matches[0][3], str)

# length
check_type(processed_matches[0][4], time)

# traits
check_type(processed_matches[0][5], list)

# units
check_type(processed_matches[0][6], list)

#### Manipulate data into necessary format

Finally the data must be output to a processed folder, ready for use:

In [16]:
# Output the players data to a CSV file
with open(players_processed_csv_path, 'w', newline='', encoding='utf-8') as players_csv_file:
    writer = csv.writer(players_csv_file)
    writer.writerow(players_cols)
    writer.writerows(players)

In [17]:
# Output the matches data to a CSV file
with open(matches_processed_csv_path, 'w', newline='', encoding='utf-8') as matches_csv_file:
    writer = csv.writer(matches_csv_file)
    writer.writerow(matches_cols)
    writer.writerows(processed_matches)